In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Generative AI Document Summarization

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/terraform-genai-doc-summarization/blob/main/notebook/gen_ai_jss.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/terraform-genai-doc-summarization/blob/main/notebook/gen_ai_jss.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>                                                                                           
</table>

**_NOTE_**: This notebook has been tested in the following environment:

* Python version = 3.8.16 (local)

## Overview

This notebook is a companion to the [Generative AI Document Summarization Jump Start Solution](https://cloud.google.com/architecture/ai-ml/generative-ai-document-summarization). With this notebook, you can use the summarization solution to create summaries of academic PDF files. In the notebook, you will programmatically upload a PDF file to a Cloud Storage bucket and then view the summary of that PDF in a BigQuery table. 

+ Learn more about [using text chat LLM with Vertex AI](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/overview).
+ Learn more about [querying tables in Cloud BigQuery](https://cloud.google.com/bigquery/docs/tables).
+ Learn more about [creating EventArc triggers for Cloud Functions](https://cloud.google.com/functions/docs/calling/eventarc).
+ Learn more about [storing data in Cloud Storage](https://cloud.google.com/storage/docs/uploading-objects).
+ Learn more about [transcribing PDFs with Cloud Vision OCR](https://cloud.google.com/vision/docs/pdf).

### Objective

In this tutorial, you learn how to trigger a Cloud Function process that transcribes characters from a PDF, stores the complete PDF text in a Storage bucket, summarizes the PDF, and then upserts the document data (summary, complete text, URI) into a BigQuery table.

This tutorial uses the following Google Cloud services and resources:

- Vertex AI Generative AI
- CLoud BigQuery
- Cloud Vision OCR
- Cloud EventArc triggers
- Cloud Functions
- Cloud Storage

The steps performed include:

- Trigger an EventArc event by uploading a PDF to a Cloud Storage bucket
- Query the BigQuery table to see the results of the summarization process

### Dataset

This notebook uses a [Kaggle dataset](https://www.kaggle.com/datasets/Cornell-University/arxiv) that contains a large collection of academic summaries from [arXiv.org](https://arxiv.org/). This dataset is made publicly available through a Cloud Storage bucket.

### Costs 

This tutorial uses billable components of Google Cloud:

* Vertex AI
* BigQuery
* Vision
* Cloud Functions
* EventArc
* Cloud Storage

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing),
and [BigQuery pricing](https://cloud.google.com/bigquery/pricing),
and [Cloud Vision pricing](https://cloud.google.com/vision/pricing),
and [Cloud Functions pricing](https://cloud.google.com/functions/pricing),
and [Cloud EventArc pricing](https://cloud.google.com/eventarc/pricing),
and [Cloud Storage pricing](https://cloud.google.com/storage/pricing), 
and use the [Pricing Calculator](https://cloud.google.com/products/calculator/?hl=en_US&_ga=2.16285759.-826855678.1689377111#id=78888c9b-02ac-4130-9327-fecd7f4cfb11)
to generate a cost estimate based on your projected usage.

## Installation

Install the following packages required to execute this notebook. 

In [5]:
%%writefile requirements.txt

google-cloud-aiplatform
google-cloud-bigquery
google-cloud-logging
google-cloud-storage
google-cloud-vision
polling2
tqdm

Overwriting requirements.txt


In [6]:
# Install the packages
import os

if not os.getenv("IS_TESTING"):
    USER = "--user"
else:
    USER = ""
! pip3 install {USER} --upgrade -r requirements.txt

  Using cached google_cloud_aiplatform-1.28.1-py2.py3-none-any.whl (2.7 MB)
  Using cached google_cloud_bigquery-3.11.4-py2.py3-none-any.whl (219 kB)
  Using cached google_cloud_logging-3.6.0-py2.py3-none-any.whl (195 kB)
  Using cached google_cloud_storage-2.10.0-py2.py3-none-any.whl (114 kB)
  Using cached google_cloud_vision-3.4.4-py2.py3-none-any.whl (444 kB)
  Using cached polling2-0.5.0-py2.py3-none-any.whl (6.4 kB)
  Using cached tqdm-4.65.0-py3-none-any.whl (77 kB)
  Using cached Shapely-1.8.5.post1-cp38-cp38-macosx_10_9_x86_64.whl (1.2 MB)
  Using cached google_cloud_resource_manager-1.10.2-py2.py3-none-any.whl (321 kB)
  Using cached google_resumable_media-2.5.0-py2.py3-none-any.whl (77 kB)
  Using cached grpc_google_iam_v1-0.12.6-py2.py3-none-any.whl (26 kB)
  Using cached google_cloud_appengine_logging-1.3.1-py2.py3-none-any.whl (16 kB)
  Using cached google_cloud_audit_log-0.2.5-py2.py3-none-any.whl (12 kB)
  Using cached google_crc32c-1.5.0-cp38-cp38-macosx_10_9_x86_64.wh

### Restart the kernel (Colab only)

In [3]:
# Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

: 

## Before you begin

### Set up your Google Cloud project

This notebook assumes that you have already deployed this solution using either the [Terraform script]() **TODO: fix target for link** or using the [Solutions console](https://console.cloud.google.com/products/solutions/catalog). During this deployment, several actions required to run this solution were performed on your behalf:

1. The [Cloud Function](https://console.cloud.google.com/functions/list) was deployed.

2. The [EventArc trigger](https://console.cloud.google.com/eventarc/triggers) was applied to the input Cloud Storage bucket.

3. The following APIs were enabled for you: 

   - [Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)
   - [BigQuery API](https://console.cloud.google.com/flows/enableapi?apiid=bigquery.googleapis.com)
   - [Cloud Vision API](https://console.cloud.google.com/flows/enableapi?apiid=vision.googleapis.com)


<div style="background-color:rgb(225,245,254); color:rgb(1,87,155); padding:2px;"><strong>Note:</strong> It is recommended to run this notebook from <a href="https://colab.sandbox.google.com/">Google Colaboratory</a>. If you are running this notebook locally instead, you need to install the <a href="https://cloud.google.com/sdk" target="_blank">Cloud SDK</a>.</div>

#### Set your project ID

**If you don't know your project ID**, try the following:
* Run `gcloud config list`.
* Run `gcloud projects list`.
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113)

In [1]:
PROJECT_ID = "[your-project-name]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

Updated property [core/project].


#### Region

You can also change the `REGION` variable used by Vertex AI. Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [2]:
REGION = "us-central1"  # @param {type: "string"}

### Authenticate your Google Cloud account

Depending on your Jupyter environment, you may have to manually authenticate. Follow one of the relevant instructions below.

**1. Colab, uncomment and run**

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

**2. Local JupyterLab instance, uncomment and run:**

In [3]:
# ! gcloud auth login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&code_challenge=9QrnfgtBYc8TN96oasecu3uRYnTwHn7vkkNYSHI8w2M&code_challenge_method=S256&access_type=offline&response_type=code&prompt=select_account



You are now logged in as [erschmid@google.com].
Your current project is [jss-16p1-test].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


**3. Service account or other**
* See how to grant Cloud Storage permissions to your service account at https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples.

### Import libraries

In [25]:
import json
import os
import polling2
import re

from datetime import datetime
from typing import Sequence, Mapping
from tqdm.notebook import tqdm
from google.cloud import aiplatform
from google.cloud import bigquery
from google.cloud import logging
from google.cloud import storage
from google.cloud import vision

import vertexai
from vertexai.preview.language_models import TextGenerationModel


## Download test data

This Jump Start Solution uses data from [arXiv.org](https://arxiv.org/) to demonstrate the summarization capabilities of Vertex AI. [Kaggle.com](https://www.kaggle.com/datasets/Cornell-University/arxiv) has made many arXiv.org scholarly papers available, free of charge, from a Google Cloud Storage bucket.

In [8]:
# List all the comparative linguistics papers from Cloud Storage
! gsutil ls gs://arxiv-dataset/arxiv/cmp-lg/pdf/9404

gs://arxiv-dataset/arxiv/cmp-lg/pdf/9404/9404001v1.pdf
gs://arxiv-dataset/arxiv/cmp-lg/pdf/9404/9404002v1.pdf
gs://arxiv-dataset/arxiv/cmp-lg/pdf/9404/9404003v2.pdf
gs://arxiv-dataset/arxiv/cmp-lg/pdf/9404/9404004v1.pdf
gs://arxiv-dataset/arxiv/cmp-lg/pdf/9404/9404005v1.pdf
gs://arxiv-dataset/arxiv/cmp-lg/pdf/9404/9404007v1.pdf
gs://arxiv-dataset/arxiv/cmp-lg/pdf/9404/9404008v1.pdf
gs://arxiv-dataset/arxiv/cmp-lg/pdf/9404/9404009v3.pdf
gs://arxiv-dataset/arxiv/cmp-lg/pdf/9404/9404010v2.pdf
gs://arxiv-dataset/arxiv/cmp-lg/pdf/9404/9404011v2.pdf


One of the available arXiv.org papers has been selected for you in the following cell. You can swap out the selected paper with another from the same source. Be sure to choose a paper that is single-column format.

In [9]:
filename = '9404002v1'
file_uri = f'gs://arxiv-dataset/arxiv/cmp-lg/pdf/9404/{filename}.pdf'

# Create a local folder and download some test PDFs
if not os.path.exists('pdfs'):
    os.mkdir('pdfs')

! gsutil cp -r $file_uri pdfs/

Copying gs://arxiv-dataset/arxiv/cmp-lg/pdf/9404/9404002v1.pdf...
/ [1 files][196.1 KiB/196.1 KiB]                                                
Operation completed over 1 objects/196.1 KiB.                                    


## Upload test data to Storage bucket

The Terraform scripts for this solution applies an EventArc trigger to a Cloud Storage bucket. When a PDF is uploaded to the storage bucket, the EventArc trigger fires, starting the summarization process.

In [10]:
INPUT_BUCKET = f'{PROJECT_ID}_uploads'

Running the following cell uploads a local PDF file (downloaded in the previous section) to the target Cloud Storage bucket. 

In [12]:
file_complete_text = f'{filename}_summary.txt'
pdf = f'pdfs/{filename}.pdf'

storage_client = storage.Client(project=PROJECT_ID)
bucket = storage_client.bucket(INPUT_BUCKET)
blob = bucket.blob(pdf)
blob.upload_from_filename(pdf)

This upload process kicks off the summarization process. You can view the progress of the summarization process in the [Logs Explorer](https://console.cloud.google.com/logs/query for your project).

To filter the logs, click the **Log name** drop-down menu and type "summarization-by-llm". Select the "summarization-by-llm" logger in the menu and then click apply to close the drop-down.

## Optional: View summarization process in Cloud Logging

You can view the results of the summarization Cloud Function as it writes updates to Cloud Logging. Each run of the summarization pipeline is associated with a `cloud_event_id`. By filtering for this ID, you can track the summarization process.

In [13]:
logger_name = 'summarization-by-llm'

@polling2.poll_decorator(check_success=lambda x: x != '', step=0.5, timeout=90)
def get_cloud_event_id(pdf_filename, bar):
    logging_client = logging.Client(project=PROJECT_ID)
    logger = logging_client.logger(logger_name)
    
    pattern = 'cloud_event_id\((.*)\):'
    cloud_id = ''
    for entry in logger.list_entries(filter_=pdf_filename, max_results=100):
        entry_text = entry.payload
        res = re.search(pattern, entry_text)
        if res != None:
            cloud_id = res.group(1)
            print(cloud_id)
            bar.update(100)
    
        if cloud_id != '':
            return cloud_id
    return cloud_id

In [14]:
with tqdm(total=100) as bar:
    cloud_event_id = get_cloud_event_id(filename, bar)
    bar.close()

  0%|          | 0/100 [00:00<?, ?it/s]

jss-16p1-test_uploads/pdfs/9404002v1.pdf/1690146389113776


Now that we have the `cloud_event_id`, we can filter for this cloud event.

In [15]:
print(f'cloud_event_id: {cloud_event_id}')

cloud_event_id: jss-16p1-test_uploads/pdfs/9404002v1.pdf/1690146389113776


In [16]:
@polling2.poll_decorator(step=10, timeout=70)
def get_cloud_event_logs(cloud_event_id):
    print("polling")
    logging_client = logging.Client(project=PROJECT_ID)
    logger = logging_client.logger(logger_name)
    
    entries = []
    for entry in logger.list_entries(filter_=cloud_event_id, max_results=100):
        entry_text = entry.payload
        entries.append(entry_text)
    return entries

In [17]:
entries = []
bar = tqdm(total=6)

for _ in range(6):
    tmp_entries = get_cloud_event_logs(cloud_event_id)
    for e in tmp_entries:
        if e not in entries:
            bar.update(1)
            entries.append(e)
            print(e)
            
    

  0%|          | 0/6 [00:00<?, ?it/s]

polling
cloud_event_id(jss-16p1-test_uploads/pdfs/9404002v1.pdf/1690146389113776): UPLOAD gs://jss-16p1-test_uploads/pdfs/9404002v1.pdf
cloud_event_id(jss-16p1-test_uploads/pdfs/9404002v1.pdf/1690146389113776): OCR  gs://jss-16p1-test_uploads/pdfs/9404002v1.pdf
cloud_event_id(jss-16p1-test_uploads/pdfs/9404002v1.pdf/1690146389113776): FULLTEXT_UPLOAD summaries/pdfs/9404002v1_fulltext.txt
cloud_event_id(jss-16p1-test_uploads/pdfs/9404002v1.pdf/1690146389113776): SUMMARY_COMPLETE
cloud_event_id(jss-16p1-test_uploads/pdfs/9404002v1.pdf/1690146389113776): SUMMARY_UPLOAD <function upload_to_gcs at 0x3ee349d06a70>
cloud_event_id(jss-16p1-test_uploads/pdfs/9404002v1.pdf/1690146389113776): DB_WRITE
polling
polling
polling
polling
polling


## Query the BigQuery table to see the summary

Once the summarization flow has completed, the summary of the PDF document should be available for you to read. To get the summary of the PDF document, you can query the BigQuery table that contains the summary.

If you do not get a result the first time you run the query, then the summarization pipeline might still be running. You might need to wait a minute to allow the pipeline to finish and to try the query again.

In [18]:
bigquery_client = bigquery.Client(project=PROJECT_ID)

table_name = f"{PROJECT_ID}.summary_dataset.summary_table"

# Compose the SQL query to select the summary for the PDF document
sql_query = f"SELECT summary FROM `{table_name}` WHERE filename LIKE '%{file_complete_text}%'"

job = bigquery_client.query(sql_query)
rows = job.result()
row_list = list(rows)

if len(row_list) != 0:
    summary = row_list[0]

print(summary['summary'])

'The Loebner prize competition is not a good way to measure the progress of AI. The competition is not well-designed, and it is not clear what purpose it serves. The Loebner prize is also inappropriate given the current level of technology.'

## Optional: Run pipeline components individually

The summarization pipeline is composed of multiple independent components. There is a component that performs optical character recognition on the PDF, another that stores data in a Storage bucket, another that performs summarization with a LLM, and yet another that stores new rows into the BigQuery table.

In this section, you can run each component individually to understand how they work together.

### Perform OCR with Cloud Vision

The first component in the pipeline performs optical character recognition (OCR) using Cloud Vision. Run the following cells to run optical character recognition on the PDF file you downloaded previously.

Note that OCR can take a while to complete. You might need to wait for a result.

In [26]:
def document_extract(
    bucket: str,
    name: str,
    output_bucket: str,
    project_id: str,
    timeout: int = 420,
) -> str:
    """Perform OCR with PDF/TIFF as source files on GCS.

    Original sample is here:
    https://github.com/GoogleCloudPlatform/python-docs-samples/blob/main/vision/snippets/detect/detect.py#L806

    Note: This function can cause the IOPub data rate to be exceeded on a
    Jupyter server. This rate can be changed by setting the variable
    `--ServerApp.iopub_data_rate_limit

    Args:
        bucket (str): GCS URI of the bucket containing the PDF/TIFF files.
        name (str): name of the PDF/TIFF file.
        output_bucket: bucket to store output in
        timeout (int): Timeout in seconds for the request.


    Returns:
        str: the complete text
    """

    gcs_source_uri = f"gs://{bucket}/{name}"
    prefix = "ocr"
    gcs_destination_uri = f"gs://{output_bucket}/{prefix}/"
    mime_type = "application/pdf"
    batch_size = 2

    # Perform Vision OCR
    client = vision.ImageAnnotatorClient()

    feature = vision.Feature(type_=vision.Feature.Type.DOCUMENT_TEXT_DETECTION)

    gcs_source = vision.GcsSource(uri=gcs_source_uri)
    input_config = vision.InputConfig(gcs_source=gcs_source, mime_type=mime_type)

    gcs_destination = vision.GcsDestination(uri=gcs_destination_uri)
    output_config = vision.OutputConfig(
        gcs_destination=gcs_destination, batch_size=batch_size
    )

    async_request = vision.AsyncAnnotateFileRequest(
        features=[feature], input_config=input_config, output_config=output_config
    )

    operation = client.async_batch_annotate_files(requests=[async_request])

    print("OCR: waiting for the operation to finish.")
    operation.result(timeout=timeout)

    # Once the request has completed and the output has been
    # written to GCS, we can list all the output files.
    return get_ocr_output_from_bucket(gcs_destination_uri, output_bucket, project_id)


def get_ocr_output_from_bucket(gcs_destination_uri: str,
                               bucket_name: str,
                               project_id: str) -> str:
    """Iterates over blobs in output bucket to get full OCR result.

    Arguments:
        gcs_destination_uri: the URI where the OCR output was saved.
        bucket_name: the name of the bucket where the output was saved.

    Returns:
        The full text of the document
    """
    storage_client = storage.Client(project=project_id)

    match = re.match(r"gs://([^/]+)/(.+)", gcs_destination_uri)
    prefix = match.group(2)
    bucket = storage_client.get_bucket(bucket_name)

    # List objects with the given prefix, filtering out folders.
    blob_list = [
        blob
        for blob in list(bucket.list_blobs(prefix=prefix))
        if not blob.name.endswith("/")
    ]

    # Concatenate all text from the blobs
    complete_text = ""
    for output in blob_list:
        json_string = output.download_as_bytes().decode("utf-8")
        response = json.loads(json_string)

        # The actual response for the first page of the input file.
        page_response = response["responses"][0]
        annotation = page_response["fullTextAnnotation"]

        complete_text = complete_text + annotation["text"]

    return complete_text

In [27]:
bucket = "arxiv-dataset"
pdf_name = "arxiv/cmp-lg/pdf/9404/9404002v1.pdf"
output_bucket = f"{PROJECT_ID}_output"

complete_text = document_extract(bucket=bucket,
                                 name=pdf_name,
                                 output_bucket=output_bucket,
                                 project_id=PROJECT_ID)

# Entire text is long; print just first 1000 characters
print(complete_text[:1000])

OCR: waiting for the operation to finish.
arXiv:cmp-lg/9404002v1 4 Apr 1994
Lessons from a
Restricted Turing Test
Stuart M. Shieber
Aiken Computation Laboratory
Division of Applied Sciences
Harvard University
March 28, 1994
(Revision 6)
Abstract
We report on the recent Loebner prize competition inspired by Turing's
test of intelligent behavior. The presentation covers the structure of the
competition and the outcome of its first instantiation in an actual event,
and an analysis of the purpose, design, and appropriateness of such a
competition. We argue that the competition has no clear purpose, that
its design prevents any useful outcome, and that such a competition is
inappropriate given the current level of technology. We then speculate as
to suitable alternatives to the Loebner prize.
This paper is to appear in Communications of the Association for Comput-
ing Machinery, and is available from the Center for Research in Computing
Technology, Harvard University, as Technical Report TR

### Summarize with Vertex AI LLM

Next, you can send the complete text of the PDF to be summarized. Vertex AI allows you to use many different types of LLM models. In this case, you use a LLM model designed for text summarization, `text-bison@001`. You send a prediction request to Vertex AI, providing the name of the LLM you want to use. The Vertex AI service then sends the model's response back to you. In the following cells, the Python SDK for Vertex AI provides all of the helper methods and classes you need to perform this process.

Note that Vertex AI predictions have a limit of characters that can be sent in a request payload. For this reason, a heuristic is needed to isolate only certain text blocks that you need--the abstract and the conclusion.

In [28]:
def predict_large_language_model(
    project_id: str,
    model_name: str,
    temperature: float,
    max_decode_steps: int,
    top_p: float,
    top_k: int,
    content: str,
    location: str = "us-central1",
    tuned_model_name: str = "",
) -> str:
    """Predict using a Large Language Model.

    Args:
      project_id (str): the Google Cloud project ID
      model_name (str): the name of the LLM model to use
      temperature (float): controls the randomness of predictions
      max_decode_steps (int): the maximum number of decode steps
      top_p (float): cumulative probability of parameter highest vocabulary tokens
      top_k (int): number of highest propbability vocabulary tokens to keep for top-k-filtering
      content (str): the text to summarize
      location (str): the Google Cloud region to run in
      tuned_model_name (str): a tuned LLM model to use; default is none

    Returns:
      The summarization of the content
    """
    vertexai.init(
        project=project_id,
        location=location,
    )

    model = TextGenerationModel.from_pretrained(model_name)
    if tuned_model_name:
        model = model.get_tuned_model(tuned_model_name)
    response = model.predict(
        content,
        temperature=temperature,
        max_output_tokens=max_decode_steps,
        top_k=top_k,
        top_p=top_p,
    )
    return response.text

In [31]:
ABSTRACT_H1 = "abstract"
CONCLUSION_H1 = "conclusion"
ABSTRACT_LENGTH = 150 * 10  # Abstract recommended max word length * avg 10 letters long
CONCLUSION_LENGTH = 200 * 10  # Conclusion max word legnth * avg 10 letters long

def truncate_complete_text(complete_text: str) -> str:
    """Extracts the abstract and conclusion from an academic paper.

    Uses a heuristics to approximate the extent of the abstract and conclusion.
    For abstract: assumes beginning after the string `abstract` and extends for 6-7 sentences
    For conclusion: assumes beginning after the string `conclusion` and extends for 7-9 sentences

    Args:
        complete_text (str): the complete text of the academic paper

    Returns
        str: the truncated paper
    """
    complete_text = complete_text.lower()
    abstract_start = complete_text.find(ABSTRACT_H1)
    conclusion_start = complete_text.find(CONCLUSION_H1)

    abstract = complete_text[abstract_start:ABSTRACT_LENGTH]
    conclusion = complete_text[conclusion_start:]
    if len(conclusion) > CONCLUSION_LENGTH:
        conclusion = conclusion[:CONCLUSION_LENGTH]

    return f"""
    Abstract: {abstract}

    Conclusion: {conclusion}
    """

In [32]:
model_name = "text-bison@001"
temperature = 0.2
max_decode_steps = 1024
top_p = 0.8
top_k = 40

extracted_text_trunc = truncate_complete_text(complete_text=complete_text)
content = f"Summarize:\n{extracted_text_trunc}",

summary = predict_large_language_model(
    project_id=PROJECT_ID,
    model_name=model_name,
    temperature=temperature,
    top_p=top_p,
    top_k=top_k,
    max_decode_steps=max_decode_steps,
    content=content)

print(summary)

The Loebner Prize competition is inspired by Turing's test of intelligent behavior.
The authors argue that the competition has no clear purpose, that its design prevents any useful outcome, and that such a competition is inappropriate given the current level of technology.
They then speculate as to suitable alternatives to the Loebner prize.


### Store summary in Cloud Storage

The output from multiple steps in the summarization process are stored in Cloud Storage. The following cells saves summarization text as a TXT file in a Storage bucket.

In [35]:
def upload_to_gcs(project: str, bucket: str, name: str, data: str):
    """Upload a string to Google Cloud Storage bucket.

    Args:
      bucket (str): the name of the Storage bucket. Do not include "gs://"
      name (str): the name of the file to create in the bucket
      data (str): the data to store

    """
    client = storage.Client(project=project)
    bucket = client.get_bucket(bucket)
    blob = bucket.blob(name)
    blob.upload_from_string(data)


In [36]:
summary_text_filename = "summaries/manual.txt"

upload_to_gcs(project=PROJECT_ID, bucket=output_bucket, name=summary_text_filename, data=summary)

### Upsert data into BigQuery

Now that you have the summary for the file, you can update the BigQuery table that contains all of the file summaries.

The following cells updates the BigQuery table in your project named `summary_dataset.summary_table` with the summaries created by Vertex LLM.

In [43]:
def write_summarization_to_table(
    project_id: str,
    dataset_id: str,
    table_id: str,
    bucket: str,
    filename: str,
    complete_text: str,
    complete_text_uri: str,
    summary: str,
    summary_uri: str,
    timestamp: datetime,
) -> Sequence[Mapping]:
    """Updates the BigQuery table with the document summarization

    Original sample is here:
    https://cloud.google.com/bigquery/docs/samples/bigquery-table-insert-rows-explicit-none-insert-ids

    Args:
      project_id (str): the Google Cloud project ID
      dataset_id (str): the name of the BigQuery dataset
      table_id (str): the name of the BigQuery table
      bucket (str): the name of the bucket with the PDF
      filename (str): path of PDF relative to bucket root
      complete_text (str): the complete text of the PDF
      complete_text_uri (str): the Storage URI of the complete TXT document
      summary (str): the text summary of the document
      summary_uri (str): the Storage URI of the summary TXT document
      timestamp (datetime): when the processing occurred
    """
    client = bigquery.Client(project=project_id)

    table_name = f"{project_id}.{dataset_id}.{table_id}"

    rows_to_insert = [
        {
            "bucket": bucket,
            "filename": filename,
            "extracted_text": complete_text,
            "summary_uri": summary_uri,
            "summary": summary,
            "complete_text_uri": complete_text_uri,
            "timestamp": timestamp.isoformat(),
        }
    ]

    errors = client.insert_rows_json(
        table_name, rows_to_insert, row_ids=bigquery.AutoRowIDs.GENERATE_UUID
    )
    if errors != []:
        logging_client = logging.Client()
        logger = logging_client.logger(logger_name)
        logger.log(
            f"Encountered errors while inserting rows: {errors}", severity="ERROR"
        )
        return errors

    return []

In [44]:
dataset_id = "summary_dataset"
table_id = "summary_table"
bucket = "gs://arxiv-dataset"

errors = write_summarization_to_table(
    project_id=PROJECT_ID,
    dataset_id=dataset_id,
    table_id=table_id,
    bucket=bucket,
    filename=pdf_name,
    complete_text=complete_text,
    complete_text_uri="",
    summary=summary,
    summary_uri=f"gs://{output_bucket}/{summary_text_filename}",
    timestamp=datetime.now(),
)

Finally, you can query the BigQuery table to ensure that the PDF summary has been inserted into the table.

In [46]:
bigquery_client = bigquery.Client(project=PROJECT_ID)

table_name = f"{PROJECT_ID}.summary_dataset.summary_table"

# Compose the SQL query to select the summary for the PDF document
sql_query = f"SELECT summary FROM `{table_name}` WHERE filename LIKE '%{pdf_name}%'"

job = bigquery_client.query(sql_query)
rows = job.result()
row_list = list(rows)

if len(row_list) != 0:
    summary = row_list[0]

print(summary['summary'])

The Loebner Prize competition is inspired by Turing's test of intelligent behavior.
The authors argue that the competition has no clear purpose, that its design prevents any useful outcome, and that such a competition is inappropriate given the current level of technology.
They then speculate as to suitable alternatives to the Loebner prize.
